# Resistor Cube Solutions

Quick symbolic and numeric solutions to the popular resistor-cube problem. 

In [28]:
import sympy as sp
import numpy as np

In [29]:
IR,v1,v2,v3,v4,v5,v6,v7 = sp.symbols('IR,v1,v2,v3,v4,v5,v6,v7')

## Circuit Matrix

After a half-page or so of algebra, we can derive the system of seven node-equations. 

$$\displaystyle \left[\begin{matrix}3 & -1 & -1 & -1 & 0 & 0 & 0\\-1 & 3 & 0 & 0 & -1 & 0 & -1\\-1 & 0 & 3 & 0 & -1 & -1 & 0\\-1 & 0 & 0 & 3 & 0 & -1 & -1\\0 & -1 & -1 & 0 & 3 & 0 & 0\\0 & 0 & -1 & -1 & 0 & 3 & 0\\0 & -1 & 0 & -1 & 0 & 0 & 3\end{matrix}\right] 
\left[\begin{matrix}V1\\V2\\V3\\V4\\V5\\V6\\V7\end{matrix}\right]
= 
\left[\begin{matrix}IR\\0\\0\\0\\0\\0\\0\end{matrix}\right]$$

We'll represent these in `sympy.Matrix` objects `A` and `b`:

In [37]:
lhs = np.array([
          [ 3,-1,-1,-1, 0, 0, 0],
          [-1, 3, 0, 0,-1, 0,-1],
          [-1, 0, 3, 0,-1,-1, 0],
          [-1, 0, 0, 3, 0,-1,-1],
          [ 0,-1,-1, 0, 3, 0, 0],
          [ 0, 0,-1,-1, 0, 3, 0],
          [ 0,-1, 0,-1, 0, 0, 3]
    ],)
A = sp.Matrix(lhs)
A

Matrix([
[ 3, -1, -1, -1,  0,  0,  0],
[-1,  3,  0,  0, -1,  0, -1],
[-1,  0,  3,  0, -1, -1,  0],
[-1,  0,  0,  3,  0, -1, -1],
[ 0, -1, -1,  0,  3,  0,  0],
[ 0,  0, -1, -1,  0,  3,  0],
[ 0, -1,  0, -1,  0,  0,  3]])

In [38]:
b = sp.Matrix([[IR],[0],[0],[0],[0],[0],[0]])
b

Matrix([
[IR],
[ 0],
[ 0],
[ 0],
[ 0],
[ 0],
[ 0]])

## Symbolic Solution

Most statements of the resistor-cube problem ask for the resistance between diagonal corners of the cube, relative to the unit-resistance per edge.  In other words, the desired quantity is: 

```
V(1) / I
```

Conveniently the `sympy.linsolve` solver will give us just this - along with all of the other node voltages:

In [32]:
sp.linsolve((A,b),(v1,v2,v3,v4,v5,v6,v7))

{(5*IR/6, IR/2, IR/2, IR/2, IR/3, IR/3, IR/3)}

These results square with our intuitive expectations. 

## Numeric Solution

Most "real" circuit-simulations are not amenable to symbolic-solvers such as `sympy`, for a slew of reasons.  

* They are often non-linear and piece-wise defined. 
* They have (many) more nodes than numeric solvers can efficiently handle. 
* Even if we generated symbolic results for complicated circuits, we'd often not know what to do with them - other than converting into numeric answers. 

Instead nearly all practical circuit simulation uses numerical solutions.  We'll just have to choose a numeric value for our single parameter `IR` - say, 1.0V.  The resistor-cube is nice enough to be completely linear, and allows us to use a linear solver such as `numpy.linalg.solve`:

In [33]:
rhs = np.transpose(np.array([1,0,0,0,0,0,0]))
np.linalg.solve(lhs, rhs)

array([0.83333333, 0.5       , 0.5       , 0.5       , 0.33333333,
       0.33333333, 0.33333333])

Again this checks out, both with our intuitive expectations and the earlier symbolic results. 

## Intuitive Solution

The resistor-cube is usually offered as a brain-teaser of sorts, whether as a job interview problem or just for shits & grins.  I have never seen anyone (but me!) work through it analytically as we have here.  

Instead, a typical thought-process looks something like so:

* We can tell from looking at this circuit that there are a few symmetries between similar-looking nodes.  This may become particularly clear when we "flatten" the cube into a 2D visual form. 
* From inspection (and our deep well of experience), we can discern that `V(2)=V(3)=V(4)`.  In other words, `I2=I3=I4 = I/3`.
* Similarly we can tell that `V(5)=V(6)=V(7)`, and that `I5=I6=I7 = I/3`. 
* With those two realizations in hand, we can further discern that each of the six middle-row resistors has *half* the current of their top and bottom-row counterparts, or `(I/3)/2 = I/6`. 
* Walking through any KVL loop including a top, middle, and bottom row resistor, we find that their voltages are `IR/3`, `IR/6`, and `IR/3`, making a total of `5*IR/6`.  The overall resistance `V(1)/I` is therefore `5*R/6`.

Although not usually asked, we can use the same insights to notice that the values of `V(5)=V(6)=V(7) = IR/3`, and that `V(2)=V(3)=V(4)` are `IR/3+IR/6 = IR/2`.  All checks out with the symbolic and numeric solutions. 